In [1]:
import numpy as np

#(1)상태와 행동, 감가율 정의
states = ["S", "R1", "R2", "R3", "F"]
actions = ["A1", "A2"]
gamma = 0.5  # 감가율

#(2)정책 π(s, a): 상태 s에서 행동 a를 선택할 확률
policy = {
    "S": {"A1": 0.6, "A2": 0.4},
    "R1": {"A1": 0.7, "A2": 0.3},
    "R2": {"A1": 1.0},
    "R3": {"A1": 1.0}
}

#(3) 상태 전이 확률 P(s, a, s'), 결정적 전이
transition_probs = {
    ("S", "A1"): "R1",
    ("S", "A2"): "R2",
    ("R1", "A1"): "R3",
    ("R1", "A2"): "R2",
    ("R2", "A1"): "R3",
    ("R3", "A1"): "F"
}

#(4) 보상 함수 R(s, a)
rewards = {
    ("S", "A1"): 0.5,
    ("S", "A2"): 1.5,
    ("R1", "A1"): 1.0,
    ("R1", "A2"): 1.5,
    ("R2", "A1"): 2.0,
    ("R3", "A1"): 3.0
}

# (5) 에피소드 시뮬레이션 함수 (정책 기반 경로)
def simulate_episode(start_state="S"):
    state = start_state
    total_return = 0
    discount = 1.0 #감가율(γ)을 누적 곱해주는 변수, 이후 단계마다 γ만큼 곱해지며 미래 보상에 대한 가치를 점점 줄인다.

    #(6) 에피소드 반복 실행
    while state != "F":
        
        #(6)-1 현재 상태에서 정책에 따라 행동 선택
        action_probs = policy[state]                     #현재 상태에서 가능한 행동들과 그에 대한 확률(policy)을 불러온다.
        actions_list = list(action_probs.keys())         #행동 이름들만 리스트로 추출한다. (예) ["A1", "A2"]
        probs = list(action_probs.values())              #행동 선택 확률만 따로 리스트로 추출한다. (예) [0.6, 0.4]
        action = np.random.choice(actions_list, p=probs) #선택은 확률적으로 이루어진다. 예: 60% 확률로 "A1", 40% 확률로 "A2" 선택

        #(6)-2 보상 받기
        reward = rewards.get((state, action), 0)         #현재 상태와 선택한 행동에 대한 보상을 불러온다.
        total_return += discount * reward                #감가율을 적용한 현재 보상을 총 보상에 더한다.

        #(6)-3 다음 상태로 이동
        next_state = transition_probs.get((state, action), "F") #현재 상태와 행동을 기반으로 다음 상태를 조회한다.
        state = next_state                                      #현재 상태를 다음 상태로 업데이트. 에이전트는 한 단계 앞으로 진행
        discount *= gamma                                       #감가율을 누적 적용한다.

    return total_return

n_episodes = 10000
state_values = {}
q_values = {}

# (7) 상태가치함수 계산: 각 상태마다 **해당 상태를 시작점**으로 하는 에피소드를 다수 실행하여, 그 상태의 기대 보상을 추정
for state in states:
    episode_returns = []

    # 여러 번 시뮬레이션 실행
    for _ in range(n_episodes):
        result = simulate_episode(start_state=state)  # 해당 상태에서 에피소드 시작
        episode_returns.append(result)                # 에피소드별 반환값 저장

    # 평균 보상을 상태가치로 저장, **각 상태별로 개별 적으로 저장**
    state_values[state] = np.mean(episode_returns)

# (8) 행동가치함수 계산
for state in policy:             #정책이 정의된 모든 상태(state)를 하나씩 꺼낸다. 예: "S", "R1", ...
    for action in policy[state]: #해당 상태에서 가능한 모든 행동(action)을 순회한다. 예: "A1", "A2"
        returns = []             #현재 상태-행동 쌍에 대해 에피소드를 여러 번 돌려 얻은 반환값을 저장할 리스트
        for _ in range(n_episodes):
            temp_state = state
            total_return = 0
            discount = 1.0

            #(8-1) 첫 행동을 강제로 선택
            reward = rewards.get((temp_state, action), 0) #첫 번째 행동은 확률이 아니라 반드시 지정된 a 를 사용해야 함
            total_return += discount * reward             #처음은 discout=1, 즉시보상 계산
            next_state = transition_probs.get((temp_state, action), "F")
            temp_state = next_state
            discount *= gamma

            #(8-2) 이후부터는 정책에 따라 행동 -> 상태가치 함수와 동일한 수식
            while temp_state != "F":
                action_probs = policy[temp_state]
                actions_list = list(action_probs.keys())
                probs = list(action_probs.values())
                next_action = np.random.choice(actions_list, p=probs) #--> 행동을 정책에 의해서 결정

                reward = rewards.get((temp_state, next_action), 0)
                total_return += discount * reward
                temp_state = transition_probs.get((temp_state, next_action), "F")
                discount *= gamma

            returns.append(total_return)
            
        # 평균 보상을 행동가치로 저장, **각 상태/행동별로 개별 적으로 저장**
        q_values[(state, action)] = np.mean(returns)

# (8) 결과 출력
print("\n✅ 상태가치함수 Vπ(s):")
for s, v in state_values.items():
    print(f"  {s}: {v:.4f}")

print("\n✅ 행동가치함수 Qπ(s,a):")
for (s, a), q in q_values.items():
    print(f"  ({s}, {a}): {q:.4f}")


✅ 상태가치함수 Vπ(s):
  S: 2.4123
  R1: 2.7191
  R2: 3.5000
  R3: 3.0000
  F: 0.0000

✅ 행동가치함수 Qπ(s,a):
  (S, A1): 1.8646
  (S, A2): 3.2500
  (R1, A1): 2.5000
  (R1, A2): 3.2500
  (R2, A1): 3.5000
  (R3, A1): 3.0000
